# 1. Menghitung HAZ-Score

## Import Library

In [ ]:
import gdown
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Understanding

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Capstone-ML/Capstone/stunting_dataset.csv')
dataset.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg)
0,0,Laki-laki,19,91.6,13.3
1,1,Laki-laki,20,77.7,8.5
2,2,Laki-laki,10,79.0,10.3
3,3,Perempuan,2,50.3,8.3
4,4,Perempuan,5,56.4,10.9


## Data Preparation

### Encoding Data Kategori

In [ ]:
label = LabelEncoder()
dataset['Jenis Kelamin'] = label.fit_transform(dataset['Jenis Kelamin'])
dataset.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg)
0,0,0,19,91.6,13.3
1,1,0,20,77.7,8.5
2,2,0,10,79.0,10.3
3,3,1,2,50.3,8.3
4,4,1,5,56.4,10.9


### Hitung HAZ-Score

In [ ]:
# # Contoh WHO Height-for-Age Z-score lookup
# Contoh WHO Height-for-Age Z-score lookup
who_haz_lookup = {
    (0, 1): (49.9, 1.9),
    (0, 0): (49.1, 1.8),
    (6, 1): (67.6, 2.6),
    (6, 0): (65.7, 2.5),
    (12, 1): (76.1, 2.9),
    (12, 0): (74.0, 2.8),
    (24, 1): (87.1, 3.1),
    (24, 0): (85.7, 3.0),
    (36, 1): (95.2, 3.5),
    (36, 0): (94.0, 3.4),
    (48, 1): (102.3, 3.8),
    (48, 0): (101.6, 3.7),
    (60, 1): (109.2, 4.2),
    (60, 0): (108.5, 4.2),
}

def calculate_haz(row):
    umur = row["Umur (bulan)"]
    jk = row["Jenis Kelamin"]
    tinggi = row["Tinggi Badan (cm)"]

    # Find the closest age group for the given gender
    closest_key = None
    min_diff = float('inf')
    for (lookup_age, lookup_jk) in who_haz_lookup.keys():
        if lookup_jk == jk:
            diff = abs(lookup_age - umur)
            if diff < min_diff:
                min_diff = diff
                closest_key = (lookup_age, lookup_jk)
            elif diff == min_diff and lookup_age < umur: # Prefer the closest age less than current if tie
                 closest_key = (lookup_age, lookup_jk)
            elif diff == min_diff and lookup_age > umur and closest_key and closest_key[0] > umur: # If closest so far is also > umur, take this one if closer
                 closest_key = (lookup_age, lookup_jk)


    if closest_key in who_haz_lookup:
        median, sd = who_haz_lookup[closest_key]
        # Handle potential division by zero if SD is 0
        if sd != 0:
            return (tinggi - median) / sd
        else:
            return None # Return None if SD is 0 to avoid error
    return None

dataset["HAZ"] = dataset.apply(calculate_haz, axis=1)

# Klasifikasi stunting
def classify_stunting(haz):
    if pd.isna(haz):
        return "Unknown"
    elif haz < -3:
        return "Severely Stunted"
    elif haz < -2:
        return "Stunted"
    else:
        return "Normal"

dataset["Stunting_Status"] = dataset["HAZ"].apply(classify_stunting)
dataset.to_csv("/content/drive/MyDrive/Capstone-ML/Capstone/stunting_dataset_with_status.csv", index=False)

dataset.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
0,0,0,19,91.6,13.3,1.966667,Normal
1,1,0,20,77.7,8.5,-2.666667,Stunted
2,2,0,10,79.0,10.3,1.785714,Normal
3,3,1,2,50.3,8.3,0.210526,Normal
4,4,1,5,56.4,10.9,-4.307692,Severely Stunted


# 2. Klasifikasi Stunting

## Data Understanding

In [ ]:
stunting = pd.read_csv('/content/drive/MyDrive/Capstone-ML/Capstone/stunting_dataset_with_status.csv')

# Display basic info and first few rows
stunting.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
0,0,0,19,91.6,13.3,1.966667,Normal
1,1,0,20,77.7,8.5,-2.666667,Stunted
2,2,0,10,79.0,10.3,1.785714,Normal
3,3,1,2,50.3,8.3,0.210526,Normal
4,4,1,5,56.4,10.9,-4.307692,Severely Stunted


In [ ]:
stunting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Id                 100000 non-null  int64  
 1   Jenis Kelamin      100000 non-null  int64  
 2   Umur (bulan)       100000 non-null  int64  
 3   Tinggi Badan (cm)  100000 non-null  float64
 4   Berat Badan (kg)   100000 non-null  float64
 5   HAZ                100000 non-null  float64
 6   Stunting_Status    100000 non-null  object 
dtypes: float64(3), int64(3), object(1)
memory usage: 5.3+ MB


## Encoding Data

In [ ]:
label = LabelEncoder()
stunting['Stunting_Status'] = label.fit_transform(stunting['Stunting_Status'])
stunting.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
0,0,0,19,91.6,13.3,1.966667,0
1,1,0,20,77.7,8.5,-2.666667,2
2,2,0,10,79.0,10.3,1.785714,0
3,3,1,2,50.3,8.3,0.210526,0
4,4,1,5,56.4,10.9,-4.307692,1


## Feature *Scaling*

In [ ]:
fitur = ['Jenis Kelamin', 'Umur (bulan)', 'Tinggi Badan (cm)', 'Berat Badan (kg)']

scaler = StandardScaler()
stunting[fitur] = scaler.fit_transform(stunting[fitur])

stunting.sample(10)


,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
686,686,1.003586,-1.387929,-0.935908,-1.108609,6.631579,0
26020,26020,-0.996426,1.389990,0.991775,0.739448,-0.433333,0
71465,71465,-0.996426,-1.387929,-1.323205,-0.926833,5.000000,0
37873,37873,-0.996426,-0.693449,-0.002875,-1.108609,2.960000,0
98118,98118,-0.996426,0.139927,0.824533,-0.442096,3.035714,0
35775,35775,-0.996426,-1.110137,-0.821480,0.345600,-0.760000,0
44082,44082,1.003586,0.139927,0.173170,0.860632,-0.344828,0
23717,23717,-0.996426,-1.110137,-1.296799,-1.593345,-2.920000,2
46953,46953,1.003586,0.973302,1.247039,0.103232,0.064516,0
43012,43012,-0.996426,-0.554553,-0.812677,-0.078544,-0.720000,0


## Data Splitting

In [ ]:
# 5. Pilih fitur dan target
X = stunting[["Jenis Kelamin", "Umur (bulan)", "Tinggi Badan (cm)", "Berat Badan (kg)"]]
y = stunting["Stunting_Status"]

# 6. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modelling

In [ ]:
# 8. Model klasifikasi
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 9. Evaluasi model
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# 10. Membuat DataFrame hasil prediksi
df_prediksi = X_test.copy()
df_prediksi["Actual_Label"] = label.inverse_transform(y_test)
df_prediksi["Predicted_Label"] = label.inverse_transform(y_pred)

# 11. Tampilkan 10 hasil prediksi pertama
df_prediksi.head(10)

Classification Report:
                   precision    recall  f1-score   support

          Normal       1.00      1.00      1.00     16949
Severely Stunted       1.00      1.00      1.00      1231
         Stunted       1.00      1.00      1.00      1820

        accuracy                           1.00     20000
       macro avg       1.00      1.00      1.00     20000
    weighted avg       1.00      1.00      1.00     20000

Confusion Matrix:
 [[16949     0     0]
 [    0  1231     0]
 [    0     0  1820]]


,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),Actual_Label,Predicted_Label
75721,1.003586,-1.526825,-1.631283,-0.684465,Normal,Normal
80184,1.003586,-0.554553,-0.627831,0.890928,Normal,Normal
19864,-0.996426,1.112198,1.484698,0.497080,Normal,Normal
76699,-0.996426,0.139927,0.815731,0.103232,Normal,Normal
92991,-0.996426,-0.137865,-0.266941,1.193889,Normal,Normal
76434,-0.996426,0.556614,0.604478,0.739448,Normal,Normal
84004,1.003586,-0.137865,-0.284545,0.769744,Stunted,Stunted
80917,-0.996426,-0.832345,-0.619029,-0.805649,Normal,Normal
60767,-0.996426,-1.110137,-1.173568,-0.139136,Stunted,Stunted
50074,1.003586,0.834406,0.085148,1.527145,Normal,Normal


In [ ]:
# Model TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 kelas: Normal, Stunted, Severely Stunted
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

# Evaluation
model.evaluate(X_test, y_test)
model.save("/content/drive/MyDrive/Capstone-ML/Capstone/model_stunting.h5")

Epoch 1/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.8422 - loss: 0.4423 - val_accuracy: 0.8769 - val_loss: 0.2775
Epoch 2/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.8848 - loss: 0.2646 - val_accuracy: 0.9075 - val_loss: 0.2177
Epoch 3/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9119 - loss: 0.2110 - val_accuracy: 0.9251 - val_loss: 0.1840
Epoch 4/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9285 - loss: 0.1768 - val_accuracy: 0.9354 - val_loss: 0.1603
Epoch 5/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9383 - loss: 0.1563 - val_accuracy: 0.9398 - val_loss: 0.1435
Epoch 6/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9404 - loss: 0.1423 - val_accuracy: 0.9438 - val_loss: 0.1277
Epoch 7/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9474 - loss: 0.1280 - val_accuracy: 0.9491 - val_loss: 0.1126
Epoch 8/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9509 - loss: 0

In [ ]:
def predict_stunting(model, scaler, input_data):
    """
    input_data: dict seperti
        {
            "Umur (bulan)": 24,
            "Tinggi Badan (cm)": 85.0,
            "Berat Badan (kg)": 12.5,
            "JK_Code": 1
        }
    """
    df_input = pd.DataFrame([input_data])
    scaled_input = scaler.transform(df_input)
    pred_probs = model.predict(scaled_input)
    pred_class = tf.argmax(pred_probs, axis=1).numpy()[0]
    return label.inverse_transform([pred_class])[0]

In [ ]:
save_path = '/content/Model'
tf.saved_model.save(model, save_path)

In [ ]:
!pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 3.32.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [ ]:
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  --output_format=tfjs_graph_model \
  '/content/Model'\
  '/content/drive/MyDrive/Capstone-ML/Capstone/Model-tfjs'

2025-05-22 17:09:53.890999: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747933793.917451    5799 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747933793.924620    5799 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
2025-05-22 17:10:03.011743: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1747933803.174835    5799 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00

## Inference Model

In [ ]:
# Load model dari file .h5
model = load_model('/content/drive/MyDrive/Capstone-ML/Capstone/model_stunting.h5')

# Cek ringkasan arsitektur
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 245 (984.00 B)

 Trainable params: 243 (972.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
predict_data = pd.read_csv('/content/drive/MyDrive/Capstone-ML/Capstone/Pemeriksaan/data_pemeriksaan.csv')
predict_data.head(10)

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date
0,1,Muhammad Habibie,22/09/2019,Laki-laki,35,10.3,81.0,2022-02-01
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,Laki-laki,16,10.0,75.0,2022-02-01
2,3,Joenathan Ivander Kusuma,25/04/2021,Laki-laki,9,9.0,76.0,2022-02-01
3,4,Trikas Wahyudi,20/03/2019,Laki-laki,34,12.5,96.0,2022-02-01
4,5,Muhammad Ihsan Fahmi,06/12/2019,Laki-laki,26,10.4,88.0,2022-02-01
5,6,Azqila Mauza Andinitya,24/10/2018,Perempuan,39,21.1,91.0,2022-02-01
6,7,Putri Nadya Silvana,01/10/2019,Perempuan,30,11.9,90.0,2022-02-01
7,8,Dea Fiedella,29/08/2021,Perempuan,6,6.9,65.0,2022-02-01
8,9,Alby Yafie Naditik,06/04/2019,Laki-laki,33,12.5,93.0,2022-02-01
9,10,Angel Almaira,12/06/2019,Perempuan,32,10.7,79.0,2022-02-01


In [ ]:
predict_data['Gender'] = label.fit_transform(predict_data['Gender'])
predict_data.head(10)

,No_Induk,Name,DOB,Gender,Age,Weight,Height,Checkup_date,HAZ,Stunting_Status
0,1,Muhammad Habibie,22/09/2019,0,35,10.3,81.0,2022-02-01,None,Unknown
1,2,Kevin Adhitama Al/Ghifary,30/10/2020,0,16,10.0,75.0,2022-02-01,None,Unknown
2,3,Joenathan Ivander Kusuma,25/04/2021,0,9,9.0,76.0,2022-02-01,None,Unknown
3,4,Trikas Wahyudi,20/03/2019,0,34,12.5,96.0,2022-02-01,None,Unknown
4,5,Muhammad Ihsan Fahmi,06/12/2019,0,26,10.4,88.0,2022-02-01,None,Unknown
5,6,Azqila Mauza Andinitya,24/10/2018,1,39,21.1,91.0,2022-02-01,None,Unknown
6,7,Putri Nadya Silvana,01/10/2019,1,30,11.9,90.0,2022-02-01,None,Unknown
7,8,Dea Fiedella,29/08/2021,1,6,6.9,65.0,2022-02-01,None,Unknown
8,9,Alby Yafie Naditik,06/04/2019,0,33,12.5,93.0,2022-02-01,None,Unknown
9,10,Angel Almaira,12/06/2019,1,32,10.7,79.0,2022-02-01,None,Unknown


In [ ]:
data_test = predict_data.copy()
fitur = ['Gender', 'Age', 'Height', 'Weight']
scaler = StandardScaler()
data_test[fitur] = scaler.fit_transform(data_test[fitur])
data_test.head(10)

NameError: name 'predict_data' is not defined